In [1]:
import pandas as pd
import re
pd.set_option('display.max_rows' , 200)

In [2]:
company_name = 'mastek'.upper()

In [3]:
particulars = 'particulars'

In [4]:
share_price_in = 1000000;

In [5]:
def calculate_growth(initial , final) :
    if not initial :
        return 1
    return round(((final - initial) / initial) * 100,2)

In [6]:
def get_share_price_in(price) :
    return round((price / share_price_in),2)

    

In [7]:
def vertical_analysis(item , divider) :
    return item/divider
    

In [8]:
def round_off_to(num , limit) :
    return round(num,limit)

In [9]:
def count_leading_space(s): 
    match = re.search(r"^\s*", s) 
    return 0 if not match else match.end()


In [10]:
def getColumnFormatterbyPercentage(type) :
    columns_perc_formats = {}
    if type == 'income_statement' :
        for column in income_statement.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    elif type == 'balance_sheet' :
        for column in balance_sheet.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    elif type == 'cashflow_statement' :
        for column in cashflow_statement.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    return columns_perc_formats

In [11]:
def styles(color = 'white',backcolor = '#308D46'):
    return [dict(selector="caption",
                       props=[("text-align", "center"),
                              ("font-size", "150%"),
                              ("color", color),
                              ("background", backcolor),
                              ("font-weight", '600'),
                              ("text-transform", 'uppercase')
                             ])]

In [12]:
income_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-income.xls')
balance_sheet = pd.read_excel('data/'+company_name+'/'+company_name+'-balance.xls')
cashflow_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-cashflow.xls')

In [13]:
start_col = income_statement.columns[1];
start_col_balance_sheet = balance_sheet.columns[1];
start_col_cashflow = cashflow_statement.columns[1];

In [14]:
income_statement.rename(columns={company_name+'_income-statement_Annual_As_Originally_Reported' : particulars} , inplace=True)
balance_sheet.rename(columns={company_name+'_balance-sheet_Annual_As_Originally_Reported' : particulars} , inplace=True)
cashflow_statement.rename(columns={company_name+'_cash-flow_Annual_As_Originally_Reported' : particulars} , inplace=True)

income_statement_temp_columns = income_statement.columns.values
income_statement_temp_columns[0] = particulars
income_statement.columns = income_statement_temp_columns

balance_sheet_temp_columns = balance_sheet.columns.values
balance_sheet_temp_columns[0] = particulars
balance_sheet.columns = balance_sheet_temp_columns

cashflow_statement_temp_columns = cashflow_statement.columns.values
cashflow_statement_temp_columns[0] = particulars
cashflow_statement.columns = cashflow_statement_temp_columns

In [15]:
basic_eps_filter = income_statement[particulars].eq('Basic EPS')
basic_eps_pos = income_statement[basic_eps_filter].index[0]

In [16]:
WASO_filter = income_statement[particulars].isin(['Basic Weighted Average Shares Outstanding','Diluted Weighted Average Shares Outstanding','Basic WASO','Diluted WASO'])
waso_pos = income_statement[WASO_filter].index

In [17]:
income_statement.dropna(how='all',subset=income_statement.columns[1:], inplace=True)
balance_sheet.dropna(how='all',subset=balance_sheet.columns[1:], inplace=True)
cashflow_statement.dropna(how='all',subset=cashflow_statement.columns[1:], inplace=True)
income_statement.fillna(0 , inplace=True)
balance_sheet.fillna(0 , inplace=True)
cashflow_statement.fillna(0 , inplace=True)

In [18]:
temp_df = income_statement.loc[0:basic_eps_pos,start_col:].copy()
temp_df = pd.DataFrame(temp_df)


In [19]:
temp_df = temp_df.apply(get_share_price_in)

In [20]:
income_statement.loc[0:basic_eps_pos,start_col:] = temp_df

In [21]:
income_statement.loc[waso_pos,start_col:] = income_statement.loc[waso_pos,start_col:].apply(get_share_price_in)

In [22]:
temp_revenue = income_statement.loc[1]
income_statement.loc[1] = income_statement.loc[0]
income_statement.loc[0] = temp_revenue 

In [23]:
is_highlighted_indexes = pd.DataFrame(income_statement.loc[0:basic_eps_pos - 1,particulars])
is_highlighted_indexes = is_highlighted_indexes[particulars].apply(count_leading_space).isin([0,4])
is_highlighted_indexes = is_highlighted_indexes.loc[is_highlighted_indexes].index

In [24]:
balance_sheet.loc[0:,start_col_balance_sheet:] =  balance_sheet.loc[0:,start_col_balance_sheet:].applymap(get_share_price_in)

In [25]:
total_equity_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Equity')
total_equity_row = balance_sheet.loc[total_equity_filter,start_col_balance_sheet:].index.values[0]
total_assets_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Assets')
total_assets_row = balance_sheet.loc[total_assets_filter,start_col_balance_sheet:]
total_liabilities_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Liabilities')
total_liabilities_row = balance_sheet.loc[total_liabilities_filter,start_col_balance_sheet:].index.values[0]
balance_sheet.loc[total_liabilities_filter,start_col_balance_sheet:] = balance_sheet.loc[[total_equity_row,total_liabilities_row],start_col_balance_sheet:].sum().values

In [26]:
cashflow_statement.loc[0:,start_col_cashflow:] =  cashflow_statement.loc[0:,start_col_cashflow:].applymap(get_share_price_in)

In [27]:
vertical_analysis_is = pd.DataFrame(columns=income_statement.columns) 
vertical_analysis_is[particulars] = income_statement.loc[0:basic_eps_pos - 1,particulars].copy()

In [28]:
total_revenue_filter = income_statement[particulars].apply(lambda x: x.strip()).eq('Total Revenue')
total_revenue = income_statement.loc[total_revenue_filter,start_col:]

In [29]:
for column in vertical_analysis_is.columns[1:] : 
    vertical_analysis_is[column] = income_statement.loc[0:basic_eps_pos - 1,column].apply(vertical_analysis,divider=total_revenue[column])
vertical_analysis_is.loc[0,particulars] = vertical_analysis_is.loc[0,particulars].strip() 

In [30]:
def row_style(row , list , key , color,bgcolor):
   if row[key] in list :
       return pd.Series('color:'+color+';background-color:'+bgcolor, row.index)
   else :
        return pd.Series('', row.index)

In [31]:
fil = vertical_analysis_is.loc[0:basic_eps_pos - 1,particulars].apply(count_leading_space).eq(0)
templist = vertical_analysis_is.loc[fil,particulars].values
vertical_analysis_is.loc[is_highlighted_indexes].style.apply(row_style,list=templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " income statement vertical analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('income_statement'),precision=2) 

,particulars,2014,2015,2016,2017,2018,2019,2020,2021,2022,TTM
0,Total Revenue,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%
1,Gross Profit,38.86%,40.96%,46.79%,43.58%,40.67%,42.60%,48.43%,50.96%,52.18%,0.00%
4,Cost of Revenue,-61.14%,-59.04%,-53.21%,-56.42%,-59.33%,-57.40%,-51.57%,-49.04%,-47.82%,0.00%
7,Operating Income/Expenses,-32.14%,-37.14%,-43.36%,-36.12%,-30.75%,-32.26%,-37.99%,-31.60%,-32.97%,-10.02%
8,"Selling, General and Administrative Expenses",-19.38%,-23.38%,-31.03%,-25.52%,-21.42%,-23.86%,-27.71%,-25.84%,-26.36%,0.00%
19,Operation and Maintenance Expenses,-2.22%,-2.02%,-2.84%,-1.83%,-1.19%,-1.02%,-1.08%,-0.76%,-0.88%,0.00%
20,"Depreciation, Amortization and Depletion",-3.57%,-3.68%,-3.04%,-2.29%,-2.30%,-1.68%,-2.32%,-2.61%,-1.96%,-2.18%
24,Provisions,0.17%,-0.70%,0.46%,0.01%,-0.11%,-1.05%,-1.93%,0.20%,-0.52%,0.00%
27,"Other Income/Expense, Operating",-7.14%,-7.35%,-6.91%,-6.49%,-5.74%,-4.65%,-4.95%,-2.59%,-3.23%,-4.59%
29,Total Operating Profit/Loss,6.73%,3.81%,3.42%,7.45%,9.92%,10.34%,10.44%,19.36%,19.22%,17.29%


In [32]:
balance_sheet_va_results_till = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Non-Controlling/Minority Interests in Equity')
balance_sheet_va_results_till = balance_sheet[balance_sheet_va_results_till].index.values[0]
bs_va_temp = balance_sheet.loc[:balance_sheet_va_results_till]
bs_va_temp = pd.DataFrame(bs_va_temp)
filt = bs_va_temp[particulars].apply(count_leading_space).isin([0,4,8])
balance_sheet_va = pd.DataFrame(bs_va_temp[filt])
bs_va_temp = None

In [33]:
for column in balance_sheet_va.columns[1:] :
    balance_sheet_va[column] = balance_sheet_va.loc[:,column].apply(vertical_analysis,divider=total_assets_row[column][0])

In [34]:
bs_fil = balance_sheet_va[particulars].apply(count_leading_space).isin([0,4])
bs_templist = balance_sheet_va.loc[bs_fil,particulars].values
balance_sheet_va.style.apply(row_style,list=bs_templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " income statement vertical analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('balance_sheet'),precision=2) 

,particulars,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Total Assets,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%
1,Total Current Assets,53.80%,56.78%,64.64%,59.66%,51.00%,49.08%,45.44%,59.69%,60.90%
2,"Cash, Cash Equivalents and Short Term Investments",22.65%,25.71%,27.92%,28.14%,24.46%,20.97%,19.66%,35.09%,31.41%
9,"Cash Restricted or Pledged, Current",0.30%,0.28%,0.09%,0.05%,0.04%,0.05%,0.03%,0.02%,0.03%
10,"Derivative Investment and Hedging Assets, Current",0.00%,0.00%,0.00%,0.00%,0.00%,0.20%,0.02%,0.00%,0.30%
11,"Trade and Other Receivables, Current",29.37%,28.63%,35.31%,30.28%,25.55%,26.98%,24.86%,23.62%,27.91%
21,"Prepayments and Deposits, Current",1.48%,2.15%,1.32%,1.19%,0.96%,0.88%,0.81%,0.88%,1.18%
22,Other Current Assets,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.06%,0.07%,0.08%
23,Total Non-Current Assets,46.20%,43.22%,35.36%,40.34%,49.00%,50.92%,54.56%,40.31%,39.10%
24,"Net Property, Plant and Equipment",9.67%,8.07%,10.20%,8.22%,5.71%,4.76%,3.84%,2.64%,2.90%
